In [ ]:
import numpy as np
import time

# Ensure local imports
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..')))
from trading_bot import trading_core

# 1. Inicializar el Manager con 4 hilos
manager = trading_core.MarketManager(4)

# 2. Definir activos de prueba y datos sintéticos
symbols = ["BTC/USDT", "ETH/USDT", "SOL/USDT", "BNB/USDT"]
n_ticks = 100  # Cantidad de datos por activo

print(f"--- Iniciando estrés test del MarketManager ---")
start_time = time.time()

# 3. Simular ráfaga de ticks (como si vinieran de WebSockets)
for i in range(n_ticks):
    for symbol in symbols:
        # Generar un precio aleatorio
        mock_price = 40000 + np.random.normal(0, 10) if "BTC" in symbol else 2000 + np.random.normal(0, 2)
        
        # Enviar al motor (esto dispara una tarea en el ThreadPool de C++)
        manager.update_tick(symbol, mock_price)

# 4. Dar un pequeño margen para que los hilos terminen de procesar
# (En un entorno real, el loop sigue vivo, aquí esperamos para validar)
time.sleep(1) 

end_time = time.time()
print(f"--- Test finalizado en {end_time - start_time:.4f} segundos ---")

# 5. Validación de integridad: ¿Podemos recuperar los últimos precios?
print("\nÚltimos precios registrados en el motor:")
for symbol in symbols:
    p = manager.get_last_price(symbol)
    print(f"{symbol}: {p:.2f}")